In [1]:
import pandas as pd
import glob
import os

# Nastav cestu k adresáři s daty
data_path = r"data"

# Najdi všechny CSV soubory s daty (ignoruj dictionary.csv)
files = glob.glob("data/Data*.csv")

# files = glob.glob(os.path.join(data_path, "Data*.csv"))

# Zkontroluj, že se soubory našly
print(f"Nalezeno {len(files)} datových souborů:")
for f in files:
    print(f)

# print(os.path.join(data_path, "Data*.csv"))
# # Načti všechny datové soubory do seznamu DataFrame
dfs = [pd.read_csv(file) for file in files]

# # Spoj všechny tabulky dohromady a přečísluj indexy
all_data = pd.concat(dfs, ignore_index=True)

# # Výstup
print("\nUkázka dat:")
print(all_data.head())


Nalezeno 7 datových souborů:
data\Data for 2019.csv
data\Data for 2020 Pandemic Year.csv
data\Data for 2021 Pandemic Year 2.csv
data\Data for 2022 End of Pandemic.csv
data\Data for 2023.csv
data\Data for 2024.csv
data\Data for 2025 Both stores as one - up to 10th oct.csv

Ukázka dat:
     År Måned  Dag Varegruppe  Antall  Beløp m/mva
0  2019     4   29  Suvenirer       2          1.0
1   NaN   NaN  Sum        NaN       2          1.0
2  2019     4   30  Suvenirer       1          0.0
3   NaN   NaN  Sum        NaN       1          0.0
4   NaN   Sum  NaN        NaN       3          1.0


In [2]:
df2 = all_data.dropna(subset=["År","Måned","Dag"])

df2.tail()


,År,Måned,Dag,Varegruppe,Antall,Beløp m/mva
51451,2025,10,10,Barn,1,219.0
51452,2025,10,10,Bøff,1,79.0
51453,2025,10,10,Pose,3,9.0
51454,2025,10,10,Luer/ull,1,499.0
51455,2025,10,10,Boxer,3,417.0


In [3]:
df2['Datum'] = pd.to_datetime(
    df2['År'].astype(str) + '-' +
    df2['Måned'].astype(str).str.zfill(2) + '-' +   # doplní nulou měsíce 1–9
    df2['Dag'].astype(str).str.zfill(2)             # doplní nulou dny 1–9
)

df2


C:\Users\anna\AppData\Local\Temp\ipykernel_6272\174317638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Datum'] = pd.to_datetime(


,År,Måned,Dag,Varegruppe,Antall,Beløp m/mva,Datum
0,2019,4,29,Suvenirer,2,1.0,2019-04-29
2,2019,4,30,Suvenirer,1,0.0,2019-04-30
5,2019,5,21,Suvenirer,1,49.0,2019-05-21
6,2019,5,21,Diverse,1,320.0,2019-05-21
8,2019,5,22,Suvenirer,3,387.0,2019-05-22
...,...,...,...,...,...,...,...
51451,2025,10,10,Barn,1,219.0,2025-10-10
51452,2025,10,10,Bøff,1,79.0,2025-10-10
51453,2025,10,10,Pose,3,9.0,2025-10-10
51454,2025,10,10,Luer/ull,1,499.0,2025-10-10


In [4]:
df2 = df2.drop(columns=['År', 'Måned', 'Dag'])

df2

,Varegruppe,Antall,Beløp m/mva,Datum
0,Suvenirer,2,1.0,2019-04-29
2,Suvenirer,1,0.0,2019-04-30
5,Suvenirer,1,49.0,2019-05-21
6,Diverse,1,320.0,2019-05-21
8,Suvenirer,3,387.0,2019-05-22
...,...,...,...,...
51451,Barn,1,219.0,2025-10-10
51452,Bøff,1,79.0,2025-10-10
51453,Pose,3,9.0,2025-10-10
51454,Luer/ull,1,499.0,2025-10-10


In [5]:
df3 = df2.rename(columns={
    "År": "Year",
    "Måned": "Month",
    "Dag": "Day",
    "Varegruppe": "Category",
    "Antall": "Quantity",
    "Beløp m/mva": "Amount"
}, inplace=False)

df3



,Category,Quantity,Amount,Datum
0,Suvenirer,2,1.0,2019-04-29
2,Suvenirer,1,0.0,2019-04-30
5,Suvenirer,1,49.0,2019-05-21
6,Diverse,1,320.0,2019-05-21
8,Suvenirer,3,387.0,2019-05-22
...,...,...,...,...
51451,Barn,1,219.0,2025-10-10
51452,Bøff,1,79.0,2025-10-10
51453,Pose,3,9.0,2025-10-10
51454,Luer/ull,1,499.0,2025-10-10


In [6]:
slovnik = pd.read_csv(r'data\dictionary.csv')

chybejici_produkty = df3[~df3['Category'].isin(slovnik['NO'])]['Category'].unique()

print("Chybějící produkty:")
print(chybejici_produkty)


Chybějící produkty:
[]


In [8]:
cz_slovnik = dict(zip(slovnik['NO'], slovnik['EN']))

df4 = df3.replace(cz_slovnik)

df4

,Category,Quantity,Amount,Datum
0,Souvenirs,2,1.0,2019-04-29
2,Souvenirs,1,0.0,2019-04-30
5,Souvenirs,1,49.0,2019-05-21
6,Various,1,320.0,2019-05-21
8,Souvenirs,3,387.0,2019-05-22
...,...,...,...,...
51451,Children,1,219.0,2025-10-10
51452,Buff,1,79.0,2025-10-10
51453,Bag,3,9.0,2025-10-10
51454,Woolen hat,1,499.0,2025-10-10


In [9]:
df5 = df4.rename(columns={
    "Datum": "Date",
    "Category": "Product_en",
    "Quantity": "Amount_pcs",
    "Amount": "Price"
})


In [10]:
# Převod ceny na float – ošetření obou variant tečka/čárka
df5['Price'] = (
    df5['Price']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

# Převod počtu kusů na integer (pokud nejsou)
df5['Amount_pcs'] = df5['Amount_pcs'].astype(int)


# Uložit výsledek do CSV vhodného pro import do SQL Serveru
df5.to_csv(
    "Sales_Fact.csv",
    sep=';',             # české CSV používá středník
    decimal=',',         # desetinná čárka místo tečky
    index=False,
    encoding='utf-8',
    float_format='%.2f'
)

print("✅ Soubor Sales_Fact.csv byl úspěšně vytvořen.")


✅ Soubor Sales_Fact.csv byl úspěšně vytvořen.


In [12]:
df5

,Product_en,Amount_pcs,Price,Date
0,Souvenirs,2,1.0,2019-04-29
2,Souvenirs,1,0.0,2019-04-30
5,Souvenirs,1,49.0,2019-05-21
6,Various,1,320.0,2019-05-21
8,Souvenirs,3,387.0,2019-05-22
...,...,...,...,...
51451,Children,1,219.0,2025-10-10
51452,Buff,1,79.0,2025-10-10
51453,Bag,3,9.0,2025-10-10
51454,Woolen hat,1,499.0,2025-10-10
